In [1]:
import pandas as pd
train_df=pd.read_csv(r'D:\mater ai\DL mini project\datasets\twitter_training.csv')
val_df=pd.read_csv(r'D:\mater ai\DL mini project\datasets\twitter_validation.csv')

In [2]:
train_df.columns=['id','entity','sentiment','tweet']
val_df.columns==['id','entity','sentiment','tweet']



array([False, False, False, False])

In [3]:
#preprocessing

import re
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')

def clean_tweet(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text=' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text
train_df['clean_tweet']=train_df['tweet'].apply(clean_tweet)
val_df_df['clean_tweet']=val_df_df['tweet'].apply(clean_tweet)

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder=LabelEncoder()
train_df['label']=encoder.fit_transform(train_df['sentiment'])


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['clean_tweet'])

sequences = tokenizer.texts_to_sequences(train_df['clean_tweet'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, train_df['label'], test_size=0.2, random_state=42)


NameError: name 'train_df' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
model = Sequential([
    Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=128, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


history = model.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_test, y_test))


In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_test, y_test))


In [ ]:
def predict_sentiment(text):
    cleaned = clean_tweet(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
    pred = model.predict(padded)[0][0]
    return "Positive 😊" if pred > 0.5 else "Negative 😠"

print(predict_sentiment("I love this movie!"))
print(predict_sentiment("This is the worst experience ever."))
